In [ ]:
%matplotlib inline
import pandas as pd
import yfinance as yf
import numpy as np
import mplfinance as mpf
import matplotlib.pyplot as plt
import talib as ta
import seaborn as sns
from datetime import datetime
sns.set_theme()

pd.set_option('display.max_columns', None);pd.set_option('display.max_rows', None)

In [ ]:
#for any ticker
ticker=input("Enter your desired ticker: ")
#ticker='SPY'
gap='1m'
days='6d'
df = yf.download(
tickers=[ticker],
period=days,
threads=True,
interval=gap,
group_by='ticker',
rounding=True)
#reset the index
df=df.reset_index()
S = pd.to_datetime(df.Datetime)

In [ ]:
S.head()

In [ ]:
S=S[::-1]

In [ ]:
S.head()

In [ ]:
def plot_with_mpl(root,tick,day=60,sty='classic',typ='candle'):
    "returns the candle stick of a stock tick for the given root file."

    df=pd.read_csv(root+'/'+tick+'.csv').tail(day)
    df['Date']=df['Date'].apply(lambda x:x.split()[0])

    df['rsi']=ta.RSI(df['Close']);df['sma-5']=ta.SMA(df['Close'],5);df['sma-10']=ta.SMA(df['Close'],10)
    #df.fillna(0,inplace=True)
    apds = [ 
         mpf.make_addplot((df['rsi']),panel=1,color='r',secondary_y=True,type='line',width=0.5),
         #mpf.make_addplot((df['sma-5']),color='g',type='line',width=0.2),
         #mpf.make_addplot((df['sma-10']),color='r',type='line',width=0.2)
       ]
    df.Volume=df.Volume/10**6              #100k volume
    df.Date=pd.to_datetime(df.Date)
    df.set_index('Date',inplace=True)
    fig,axes=mpf.plot(df,type=typ,mav=(5,21),volume=True,tight_layout=False,figratio=(80,36),figscale=1,
                    style=sty,returnfig=True,addplot=apds,scale_width_adjustment=dict(volume=0.6,candle=0.8,lines=0.4))#,update_width_config=dict(candle_linewidth=1.75,candle_width=0.1))
    axes[0].legend([" "," ","5-SMA","21-SMA"],loc='upper center')
    #axes[0].legend(loc='upper left')
    axes[0].set_title(tick)
    mpf.show()

In [ ]:
df.groupby((S-S[0]).astype('timedelta64[D]'))

In [ ]:
#S[0]-S

In [ ]:
s=20*"=="
kwargs={'color':'r', 'fontsize':14}
kwargs1={'color':'b', 'linestyle':'-','lw':2}
for i, g in df.groupby([(S-S[0]).astype('timedelta64[D]')]):
    g.Volume=round(g.Volume/10**6,2);
    g['rsi']=ta.RSI(g['Close']);g['sma-5']=ta.SMA(g['Close'],5);g['sma-10']=ta.SMA(g['Close'],10)
    
    apds = [ 
         mpf.make_addplot((g['rsi']),panel=1,color='r',secondary_y=True,type='line',width=0.5),
         mpf.make_addplot((g['sma-5']),color='g',type='line',width=0.2),
         mpf.make_addplot((g['sma-10']),color='r',type='line',width=0.2)
       ]
    
    g.Datetime=pd.to_datetime(g.Datetime)
    title=g.head(1).Datetime.to_string().split()[1]
    g.set_index('Datetime',inplace=True)
    fig,axes=mpf.plot(g,type=typ,mav=(5,10),volume=True,tight_layout=False,figratio=(150,50),figscale=1,
                    style='default',returnfig=True,addplot=apds,scale_width_adjustment=dict(volume=0.6,candle=0.8,lines=0.4))#,update_width_config=dict(candle_linewidth=1.75,candle_width=0.1))
    axes[0].legend([" "," ","5-SMA","10-SMA"],loc='upper center')
    op=g.Open.values[0]
    cl=g.Close.values[-1]
    hi=g.High.max()
    lo=g.Low.min()
    vo=g.Volume.sum().round(2)
    st=f"\nopen   : {op}\
        \nlow    : {lo}\
           \nhigh   : {hi}\
         \nclose  : {cl}\
         \nvolume : {vo} mln"
    print("{:^}".format(st))
    # #axes[0].legend(loc='upper left')
    #title=g.head(1).Datetime.to_string().split()[1]
    #axes[0].legend(st,loc='upper left')
    # axes[0].set_title(title)
    # axes[0].text(0.1,0.2,'tst')
    # axes[0].text(0.7, 0.3, st, style='italic',
    #     bbox={'facecolor': 'red', 'alpha': 0.05, 'pad': 10},color='blue',fontsize=15,fontweight='bold')
    # #g['Close'].plot(ax=axes[count,count])
    mpf.show()
    #print(f"{i+1}{s:^}")
    #print(g.tail().to_string())
    #idx=g.Datetime.apply(lambda x:x.to_string().split()[2])
    # idx=g.Datetime.apply(lambda x:x.strftime('%H-%M')).to_list()
    # #print(f"idx: {idx}")
    # title=g.head(1).Datetime.to_string().split()[1]
    # #print(title)
    # ax=g.plot(x='Datetime',y='Close',rot=90,figsize=(20,10),title=title,**kwargs1)#.set_xticks(g.index,idx)#,**kwargs)
    # plt.ylabel('CP')
    # plt.xlabel('Time')
    # plt.tight_layout()
    # plt.show()

In [ ]:
sty='yahoo';typ='candle'
#fig,axs=plt.subplots(5,1)
fig, axes = plt.subplots(nrows=3, ncols=2,figsize=(20,16))
cols=axes.flat
#print(f"{cols}")
j=5
sty='yahoo';typ='candle'
#fig,axs=plt.subplots(5,1)
fig, axes = plt.subplots(nrows=3, ncols=2,figsize=(20,16))
cols=axes.flat
#print(f"{cols}")
j=5
for i, g in df.groupby([(S- S[0]).astype('timedelta64[D]')]):
    g.Volume=round(g.Volume/10**6,2);
    #fig,axes=mpf.plot(g,type=typ,mav=5,volume=True,tight_layout=False,figratio=(24,12),figscale=1,
    #                returnfig=True,style=sty)
    #print(g.head())
    #print(g.dtypes)
    #g['Adj Close'].plot()
    g.plot(x='Datetime',y='Adj Close',ax=cols[j])
    #g.plot(x='Datetime',y='Volume',ax=cols[j])
    g.plot(x='Datetime',y='Volume',secondary_y=True, ax=cols[j],mark_right=False,color='g')
    #g.plot(x='Datetime',y='Volume',secondary_y=True, ax=cols[j],kind='line',mark_right=False,color='g')
    cols[j].set_ylabel("Price ($)");
    cols[j].right_ax.set_ylabel("Volume (mln)");
    xposition=cols[j].get_xticks()
    for xc in xposition:
        cols[j].axvline(x=xc, color='r', linestyle=':')
    #par='Close'
    #print(f"{j+1})Closes:\n {g[par]}")
    op=g.Open.values[0]
    
    op=g.Open.values[0]
    cl=g.Close.values[-1]
    hi=g.High.max()
    lo=g.Low.min()
    vo=g.Volume.sum().round(2)
    st=f"\nopen   : {op}\
        \nlow    : {lo}\
           \nhigh   : {hi}\
         \nclose  : {cl}\
         \nvolume : {vo} mln"
    print(st)
    #cols[j].text(0.5,0.3,st,transform=cols[j].transAxes,color='blue',fontsize=15,fontweight='bold',horizontalalignment='left')
    cols[j].text(0.7, 0.3, st, style='italic',
        bbox={'facecolor': 'red', 'alpha': 0.05, 'pad': 10},transform=cols[j].transAxes,color='blue',fontsize=15,fontweight='bold')
    #g['Close'].plot(ax=axes[count,count])
    j-=1
    # ax=g.plot(x='Datetime',y='Adj Close',figsize=(4,3),legend=True)
    # xposition=ax.get_xticks()
    # for xc in xposition:
    #     ax.axvline(x=xc, color='r', linestyle=':')
    #print(t)
plt.tight_layout()
fig.suptitle(ticker)
#plt.savefig(f'{ticker}.pdf')
plt.show()

                                   